In [102]:
import pickle
import pandas as pd
import os
import git
import numpy as np
from tqdm import tqdm
import re
## tokenize the text..
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer as keras_Tokenizer
from sklearn.metrics import classification_report
from keras.layers import Embedding
from nltk.tokenize import TweetTokenizer
tweet_tknzr = TweetTokenizer()
## keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Concatenate
from keras.optimizers import SGD
from keras.models import Model, Input 
from keras.models import Sequential
from keras.layers import Dense, concatenate
from keras.layers import Flatten
from keras.layers import SpatialDropout1D
from keras.layers import Embedding 
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Average
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from keras.models import Model, Input

In [2]:
## Read the labelled files and the poly_user
def get_git_root(path):
	git_repo = git.Repo(path, search_parent_directories=True)
	git_root = git_repo.git.rev_parse("--show-toplevel")
	return git_root

In [7]:
top_dir = os.path.join(get_git_root(os.getcwd()))
model_dir = os.path.join(get_git_root(os.getcwd()),"models")
input_dir = os.path.join(get_git_root(os.getcwd()),"input")
poly_dir = os.path.join(model_dir,"poly_users")
embeddings_dir = os.path.join(input_dir,"embeddings")
poly_users = pickle.load(open(os.path.join(poly_dir,"poly_users.pkl"),"rb"))
regular_users = pickle.load(open(os.path.join(poly_dir,"regular_users.pkl"),"rb"))
mono_users = pickle.load(open(os.path.join(poly_dir,"mono_users.pkl"),"rb"))

In [8]:
## input data :juul_data 
juul_data = pd.read_csv(os.path.join(input_dir,"juul_data.csv"),lineterminator="\n")
juul_data["tweetCreatedAt"] = pd.to_datetime(juul_data["tweetCreatedAt"])
juul_data = juul_data.loc[juul_data.userID.isin(regular_users)]

# ## look at the weed first and juul_first tweet
first_data = pd.read_csv(os.path.join(input_dir,"user_first.csv"),lineterminator="\n")
first_data = first_data.loc[first_data.userID.isin(regular_users)]  # reg users
first_data["weed_first"] = pd.to_datetime(first_data["weed_first"])
first_data["juul_first"] = pd.to_datetime(first_data["juul_first"])

embedding_file = os.path.join(embeddings_dir,"glove.twitter.27B.100d.txt")

In [9]:
## removing the rewteet text column completely
ids_ = juul_data.loc[juul_data['retweetText'].notnull()]["tweetId"]
juul_data.loc[juul_data.tweetId.isin(ids_),"tweetText"] = juul_data.loc[juul_data.tweetId.isin(ids_)]["retweetText"]
juul_data = juul_data.drop(["hashtags","retweetText"],axis=1)
len(juul_data) ## 1353987 (sanity check)

1353987

In [10]:
len(juul_data) ## 1353987
len(juul_data.userID.unique()) ## 673724  reg users

673724

In [ ]:
## @ returns the data in that year
def get_year_data(year):
    print("year",year)
    users_ = list(first_data["userID"].loc[ 
        ((first_data.juul_first.dt.year <= year)  & (first_data.juul_first.dt.year > (year-1)))
        & ((first_data.weed_first.dt.year == (year + 1)) | (pd.isnull(first_data.weed_first))) ## weed data after 2015
                ]) # users who will change after september

    poly_turn = list(first_data["userID"].loc[
        (first_data.juul_first.dt.year <= year)  & 
        ((first_data.weed_first.dt.year == (year +1 )))]) ## for labelling based on the next year


    print("users that will change",len(poly_turn))
    print("total users",len(users_))

    ## getting the input data
    data_ = juul_data.loc[juul_data.userID.isin(users_)]
    print("length of data",len(data_))

    ## get label - they reamain same for this task as the no of users, we only change the tweets data
    users_ = data_.userID.unique()
    users_lbl = pd.DataFrame(users_,columns=["userID"])
    users_lbl["label"] = 0  ## initialize
    users_lbl.loc[users_lbl.userID.isin(poly_turn) , "label"] = 1
    len(users_lbl.loc[users_lbl.label == 1])  ## sanity check
    return ((year,data_,users_lbl))

In [119]:
## HELPER FUCNTIONS
# setup env
import nltk
from numpy import zeros
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tweet_tknzr.tokenize(sentence)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)] ## remove punctuations
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_max_length(df):
    ## max_length
    lengths = df["tweetText"].progress_apply(get_length)
    max_len = int(lengths.quantile(0.95))
    return (max_len)

def get_length(s):
    a = list(s.split())
    return(len(a))

def clean_text(text):
    text = re.sub(r'(https?://\S+)', "", text) ## remove url
    text = re.sub(r'(\@\w+)', "author",text)   ## remove @ mentions with author
    text = re.sub(r'(@)', "",text)             ## remove @ symbols
    text = re.sub(r'(author)',"",text)         ## remove author
    text = re.sub(r'(#)', "",text)             ## removing the hashtags signal 
    text = re.sub(r'(RT )', "",text)         ## remove the retweet info as they dont convey any information
    text = re.sub(r'(^:)',"",text)
    text = text.rstrip() 
    text = text.lstrip()
    return(text)

## returns the emnbedding matrix for the lstm model
def get_embedding_matrix(vocab_size,dimension,embedding_file,keras_tkzr):
    word2vec = get_word2vec(embedding_file)
    from numpy import zeros
    embedding_matrix = zeros((vocab_size, dimension))
    for word, i in keras_tkzr.word_index.items():
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/drew_william2345/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/drew_william2345/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [121]:
from sklearn.model_selection import cross_val_score

def get_baseline_scores(X_train,X_test,Y_train,Y_test):
    
    print("training the models")
    print("svm")
    svm = LinearSVC(C=1,verbose=1)
    svm.fit(X_train,Y_train)
    svm_pred = svm.predict(X_test)
    svm_score = precision_recall_fscore_support(Y_test,svm_pred,average=None)[2]  # return the f-score
    svm_f1 =  cross_val_score(svm, X_test, Y_test, cv=5,scoring='f1_macro').mean()
    print('svm cross val score mean',svm_f1,'\n')
    
    print("random_forest")
    rf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
    rf.fit(X_train,Y_train)
    rf_pred = rf.predict(X_test)
    rf_score = precision_recall_fscore_support(Y_test,rf_pred,average=None)[2]
    rf_f1  = cross_val_score(rf, X_test, Y_test, cv=5,scoring='f1_macro').mean()
    print('rf cross val score mean',rf_f1,'\n')
    
    print("xgBoost")
    xgb = XGBClassifier()
    xgb.fit(X_train, Y_train)
    xgb_pred = xgb.predict(X_test)
    xgb_score = precision_recall_fscore_support(Y_test,xgb_pred,average=None)[2]
    xgb_f1 =  cross_val_score(xgb, X_test, Y_test, cv=5,scoring='f1_macro').mean()
    print('xgb corss val score mean',xgb_f1,'\n')
    
    y_pred = [1 for x in range(len(Y_test))]
    print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')
    maj_score = precision_recall_fscore_support(Y_test,y_pred,average=None)[2]
    
    models = {0:[svm_pred,svm,"svm"], 1:[rf_pred,rf,"rf"] ,2:[xgb_pred,xgb,"xgb"]}
    model_idx = np.argmax([svm_f1,rf_f1,xgb_f1])  ## get the best performing model
    
    print("selecting the best model",models[model_idx][2])

    print("job finished")
    all_scores = {
        'svm': [svm,svm_score],
        'rf' : [rf,rf_score],
        'xg_boost': [xgb,xgb_score],
        'maj': [maj_score],
    }
    return (all_scores,models[model_idx])

def prepare_data(input_data,users_labelled):
    ## preapring the user data
    user_data = input_data.groupby(by="userID").agg({  'tweetText' : 'count',
                                                       'followersCount': 'first',
                                                       'friendsCount' : 'first',
                                                       'statusesCount' : 'first',
                                                       'favourites_count' : 'first',
                                                       'listedCount' : 'first',
                                                      })
    user_data = user_data.rename(columns={'tweetText': 'tweetCount'})
    
    # preapring text
    tweet_data = input_data.groupby(by="userID")["tweetText"].apply(lambda x: "%s" % ' '.join(x)).reset_index()
    ## cleaning the text
    tweet_data["tweetText"] = tweet_data["tweetText"].progress_apply(clean_text)
    tweet_data["tweetText"] = tweet_data["tweetText"].progress_apply(get_tokens).str.join(" ")
    
    ## merging the text and user data
    final_data = user_data.join(tweet_data.set_index("userID"),on="userID",how="inner").reset_index()
    final_data = final_data.fillna(0)   
    
    ## extract the labels
    y = list(final_data.join(users_labelled.set_index("userID"),on="userID",how="inner")["label"]) 
    print("downsampling")
    
    ## downsampling based on userIDS
    userIDs = np.array(list(final_data.userID)).reshape(-1,1)
    rus = RandomUnderSampler(random_state=0)
    rus.fit(userIDs,y)
    userIDs , y_sam = rus.fit_sample(userIDs,y)
    input_data = (final_data.loc[final_data.userID.isin(userIDs.flatten())])
    print("downsampled data length",len(input_data))
    
    print("train-test split")
    train_data,test_data,Y_train,Y_test  = train_test_split(input_data, y_sam,test_size =0.20,random_state= 4,shuffle=True ,
                                                          stratify= y_sam)
    return(train_data,test_data,Y_train,Y_test)

## return user fatures
def prepare_user_features(input_):
    user_data = input_[["followersCount","friendsCount","statusesCount"
                          ,"favourites_count","listedCount","tweetCount",]]
    ## followerss/ friends ration
    user_data["ff_ratio"] = user_data["followersCount"] / user_data["friendsCount"]

    ## using log of each of the columns
    user_data[["followersCount","friendsCount"
          ,"statusesCount","favourites_count",
          "listedCount"]] = np.log(user_data[["followersCount","friendsCount","statusesCount","favourites_count",
                                        "listedCount"]])
    
    user_data["unigrams"] = list(input_["tweetText"].apply(get_length))

    ## replace the na and inf values 
    user_data = user_data.replace([np.inf, -np.inf], np.nan)
    user_data.replace(np.nan,0)
    user_data = user_data.fillna(0)

    ## normalizing the values
    user_data = (user_data - user_data.min())/(user_data.max()-user_data.min())
    user_data = user_data.replace([np.inf, -np.inf], np.nan)
    user_data = user_data.fillna(0)

    X = user_data.values
    return (X,user_data)

def run_user_features(train_data,test_data,Y_train,Y_test):
    X_train,_ = prepare_user_features(train_data)
    X_test,_ = prepare_user_features(test_data)
    
    scores,best_model = get_baseline_scores(X_train,X_test,Y_train,Y_test)
    return(scores,best_model[0],best_model[1])

##models

## @ return a trained svm model
def svm_wrapper(X_train,Y_train):
    svm = LinearSVC(C=1, verbose=1)
    svm.fit(X_train, Y_train)
    return svm

def run_lstm(train_data,test_data,Y_train,Y_test,dimension,epoch,metrics,weight=None):
    scores = []
    ## print winodow , max_len for analysis purpose
    max_len = get_max_length(train_data)
    if max_len > 60:
        max_len = 60
    print("max_length",max_len)

    ## prepare the tokenizer
    print("preparing the tokenizer")
    keras_tkzr = keras_Tokenizer()
    keras_tkzr.fit_on_texts(train_data["tweetText"])
    vocab_size = len(keras_tkzr.word_index) + 1
    print("vocalb",vocab_size)
    
    ## embedding matrix
    print("creating glove embeddign matrix")
    embedding_matrix = get_embedding_matrix(vocab_size,dimension,embedding_file,keras_tkzr) ## tokenizer contains the vocalb info

    ## encoding the docs
    print("encoding the data")
    encoded_docs = keras_tkzr.texts_to_sequences(train_data["tweetText"])
    X_train = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))

    ## split the data
    print("train-val split")
    X_train,X_val,Y_train,Y_val  = train_test_split(X_train, Y_train,test_size =0.25,random_state= 4,shuffle=True ,
                                                     stratify= Y_train)
    ## encoding the test data
    encoded_docs = keras_tkzr.texts_to_sequences(test_data["tweetText"])
    X_test = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))
    
    print("X-train",X_train.shape)
    print("X-test",X_test.shape)

    print("creating lstm model")
    model = get_lstm_model(max_len,vocab_size,dimension,embedding_matrix)
    
    print("training the model with balance dataset")
    
    history = model.fit(X_train,Y_train,validation_data = (X_val,Y_val) , nb_epoch = epoch, 
                        verbose = 1,batch_size=32,class_weight= weight,callbacks=[metrics])

    ##plotting trainin validation - no point as we dont want ot look at accuarcy
    training_plot(history)

    kFold = StratifiedKFold(n_splits=3)
    for train, test in kFold.split(X_train, Y_train):
        scores.append(model.evaluate(X_train[test], Y_train[test])[1])  ## accuarcy

    print("generating classfication report")
    loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
    print('Accuracy: %f' % (accuracy*100))
    ## lstm model
    temp = model.predict(X_test)
    y_pred = [np.argmax(value) for value in temp]  ## sigmoid
    print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

    print("lstm cross val score ",np.array(scores).mean() )

    print("job finished")
    return (scores,y_pred,model,keras_tkzr,max_len)

## @ return a trained svm model on text features for LR
def run_text_features(train_data,test_data,Y_train,Y_test):
    tf_idf = TfidfVectorizer(sublinear_tf=True)
    tf_idf.fit(train_data["tweetText"])  ## fit on train data
    
     ## transform train and test data
    X_test = tf_idf.transform(test_data["tweetText"])
    X_train = tf_idf.transform(train_data["tweetText"])
    
    
    ## reduce the dimesionality
    svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
    svd.fit(X_train)  
    X_train = svd.transform(X_train)
    X_test = svd.transform(X_test)
    
    scores,best_model = get_baseline_scores(X_train,X_test,Y_train,Y_test)
    return (scores,best_model[0],best_model[1],tf_idf,svd)

def cal_text_pred(test_data,Y_test,model,tf_idf,svd):
    X_test = tf_idf.transform(test_data["tweetText"])
    X_test = svd.transform(X_test) ## reduce the dimensionality
    y_pred = model.predict(X_test)
    return y_pred

def cal_lstm_pred(test_data,Y_test,model,keras_tkzr,max_len):
     ## encoding the test data
    encoded_docs = keras_tkzr.texts_to_sequences(test_data["tweetText"])
    X_test = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))
    ## calculate the model predictions
    temp = model.predict(X_test)
    y_pred = [np.argmax(value) for value in temp]  ## sigmoid
    return y_pred

def cal_user_pred(test_data,Y_test,model):
    X_test,_ = prepare_user_features(test_data)
    y_pred = model.predict(X_test)
    return y_pred

# create the word2vec dict from the dictionary
def get_word2vec(file_path):
    file = open(file_path, "r")
    if (file):
        word2vec = dict()
#         split = file.read().splitlines()
        for line in file:
            split_line = line.split(' ')
            key = split_line[0] # the first word is the key
            value = np.array([float(val) for val in split_line[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid fiel path")

In [128]:
## handle two different inputs and then concatenate them (user and text features)
## input = [words_in,user_in]
def create_model(max_len,user_feature_len,vocalb_size,embedding_matrix):
    ## handle text features..
    words_in = Input(shape=(max_len,))
    emb_word = Embedding(vocab_size,100,weights=[embedding_matrix],input_length=max_len)(words_in) 
    lstm_word =  Bidirectional (LSTM (100,return_sequences=False,dropout=0.50,kernel_regularizer=regularizers.l2(0.01)),merge_mode='concat')(emb_word)
    lstm_word = Dense(30,activation='relu')(lstm_word)
    
    ## takes the user features as input
    user_input = Input(shape=(user_feature_len,))
    
    ## concatenate both of the features
    modelR = concatenate([lstm_word, user_input])
    # modelR = SpatialDropout1D(0.1)(modelR)
    model = Dense(100,activation='relu')(modelR)
    output = Dense(2,activation='softmax')(model)
    model = Model([words_in,users_In],output)
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model
    

In [88]:
## gathering data in different range of intervals
bucket_data = []
bucket_data.append(get_year_data(2015))
bucket_data.append(get_year_data(2016))
bucket_data.append(get_year_data(2017))

year 2015
users that will change 97
total users 1049
length of data 2944
year 2016
users that will change 313
total users 1636
length of data 6475
year 2017
users that will change 9525
total users 40236
length of data 111735


In [ ]:
tqdm.pandas()

year,data,users_labelled  = bucket_data[0]

dimension = 100
# prepare_data
train_data,test_data,Y_train,Y_test = prepare_data(data,users_labelled)
scores = []
## print winodow , max_len for analysis purpose
max_len = get_max_length(train_data)
if max_len > 60:
    max_len = 60
print("max_length",max_len)

## prepare the tokenizer
print("preparing the tokenizer")
keras_tkzr = keras_Tokenizer()
keras_tkzr.fit_on_texts(train_data["tweetText"])
vocab_size = len(keras_tkzr.word_index) + 1
print("vocalb",vocab_size)

## embedding matrix
print("creating glove embeddign matrix")
embedding_matrix = get_embedding_matrix(vocab_size,dimension,embedding_file,keras_tkzr) ## tokenizer contains the vocalb info

## encoding the docs
print("encoding the data")
encoded_docs = keras_tkzr.texts_to_sequences(train_data["tweetText"])
X_train = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))

## split the data
print("train-val split")
X_train,X_val,Y_train,Y_val  = train_test_split(X_train, Y_train,test_size =0.25,random_state= 4,shuffle=True ,
                                                 stratify= Y_train)
## encoding the test data
encoded_docs = keras_tkzr.texts_to_sequences(test_data["tweetText"])
X_test = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))

print("X-train",X_train.shape)
print("X-test",X_test.shape)

In [125]:
## getting the user features
X_train_user,_ = prepare_user_features(train_data)
X_test_user,_ = prepare_user_features(test_data)

user_feat_len = X_train_user.shape[1]

/home/drew_william2345/anaconda3/envs/multimodal-env/lib/python3.6/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/drew_william2345/anaconda3/envs/multimodal-env/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in log
/home/drew_william2345/.local/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [127]:
print("creating lstm model")
model = create_model(max_len,user_feature_len,vocab_size,dimension,embedding_matrix)

creating lstm model


TypeError: create_model() takes 4 positional arguments but 5 were given

In [122]:
print("training the model with balance dataset")

history = model.fit([X_train,X_test_user],Y_train,validation_split=0.25 , nb_epoch = epoch, 
                    verbose = 1,batch_size=32,class_weight= weight,callbacks=[metrics])

##plotting trainin validation - no point as we dont want ot look at accuarcy
training_plot(history)

kFold = StratifiedKFold(n_splits=3)
for train, test in kFold.split([X_train,X_test_user], Y_train):
    scores.append(model.evaluate(X_train[test], Y_train[test])[1])  ## accuarcy

print("generating classfication report")
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))
## lstm model
temp = model.predict(X_test)
y_pred = [np.argmax(value) for value in temp]  ## sigmoid
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

print("lstm cross val score ",np.array(scores).mean() )

print("job finished")
return (scores,y_pred,model,keras_tkzr,max_len)





  0%|          | 0/1049 [00:00<?, ?it/s]



100%|██████████| 1049/1049 [00:00<00:00, 51416.64it/s]



  0%|          | 0/1049 [00:00<?, ?it/s]



 29%|██▉       | 307/1049 [00:00<00:00, 3062.05it/s]



 55%|█████▌    | 582/1049 [00:00<00:00, 2959.99it/s]



 77%|███████▋  | 807/1049 [00:00<00:00, 2667.65it/s]



100%|██████████| 1049/1049 [00:00<00:00, 2577.20it/s]



  0%|          | 0/155 [00:00<?, ?it/s]



100%|██████████| 155/155 [00:00<00:00, 67112.33it/s]

downsampling
downsampled data length 194
train-test split
max_length 60
preparing the tokenizer
vocalb 2073
creating glove embeddign matrix
encoding the data
train-val split
X-train (116, 60)
X-test (39, 60)


In [ ]:
## running by year
svm_scores = []
rf_scores = []
xgb_scores = []
maj_scores = []
for year,data,users_labelled in bucket_list:
    print("************")
    print("preparing the data")
    train_data,test_data,Y_train,Y_test = prepare_data(data,users_labelled)
    print("runnning classfier to get user features")
    
    ## user features
    X_train,_ = prepare_user_features(train_data)
    X_test,_ = prepare_user_features(test_data)
    
    user_feature_len = len(X_train.shape[1])
    